# Setup

In [ ]:
!pip install yolov5 -q

from yolov5 import train

import locale
locale.getpreferredencoding = lambda: "UTF-8"

# Download dataset
**You can obtain your link (Roboflow key) on the download page [here](https://universe.roboflow.com/brad-dwyer/aquarium-combined/dataset/5/download) (requires sign-up)**

In [ ]:
# %env YOUR_ROBOFLOW_KEY=

In [ ]:
import os

if "YOUR_ROBOFLOW_KEY" not in os.environ:
  raise RuntimeError("Set YOUR_ROBOFLOW_KEY environmental variable above!")

In [ ]:
!curl -L "https://universe.roboflow.com/ds/aXGylruXWt?key=$YOUR_ROBOFLOW_KEY" > roboflow.zip;
!unzip -o -q roboflow.zip -d data; rm roboflow.zip;

**Insert path to dataset root directory**

In [ ]:
# NOTE - Update with your path if not running in Google Colab
!sed -i "1i path: /content/data" data/data.yaml

**Fix mislabeled annotation**

In [ ]:
%env mislabeled_file=data/train/labels/IMG_8590_MOV-3_jpg.rf.e215fd21f9f69e42089d252c40cc2608.txt
!awk '{print "1", $2, $3, $4, $5}' $mislabeled_file > tmp.txt && mv tmp.txt $mislabeled_file

# Training

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs/train

## Define hyperparameters

In [ ]:
import yaml
import torch

# Default hyperparameters config
hyp_file = 'hyp.scratch-low.yaml'
hyp_url = f'https://raw.githubusercontent.com/ultralytics/yolov5/master/data/hyps/{hyp_file}'

# Get default hyperparameters config
torch.hub.download_url_to_file(hyp_url, hyp_file)

# Load YAML into dict
with open(hyp_file, errors='ignore') as f:
    hyps = yaml.safe_load(f)

# MODIFY HYPERPARAMETERS
# hyps['warmup_bias_lr'] = 0.0
# hyps['lr0'] = 0.001
# hyps['obj'] = 0.1

# Dump dict into YAML file
with open(hyp_file, 'w') as f:
  yaml.dump(hyps, f, sort_keys=False)

100%|██████████| 1.65k/1.65k [00:00<00:00, 1.92MB/s]


## Choose weights

In [ ]:
# From YOLOv5 repo
weights='yolov5s.pt'

# Local
# weights='last.pt'
# weights='best.pt'

# Continue training - use weights already uploaded to HF to improve upon them
# weights = 'akbojda/yolov5s-aquarium'

## Train

In [ ]:
train.run(imgsz=640, epochs=300, data='data/data.yaml', weights=weights, logger='TensorBoard', hyp=hyp_file, cache='ram');

**Download results (for Colab experiments)**

In [ ]:
# from google.colab import files

# !zip -r results.zip /content/runs/train
# files.download('results.zip')